# Live Demo!

### Imports

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as mplplot
from mpl_toolkits.mplot3d import Axes3D

import tensorflow as tf

%matplotlib inline

### Generate data

In [ ]:
# specify points that define a helical shape in three dimensions
zCoords = np.linspace(0, 1, 1000)
yCoords = np.abs(0.5 - zCoords) * np.cos(8 * np.pi * zCoords)
xCoords = np.abs(0.5 - zCoords) * np.sin(8 * np.pi * zCoords)

sigma = 0.02

zSample = np.random.choice(zCoords, 200)
ySample = np.abs(0.5 - zSample) * np.cos(
    8 * np.pi * zSample
) + np.random.normal(0, sigma, zSample.size)
xSample = np.abs(0.5 - zSample) * np.sin(
    8 * np.pi * zSample
) + np.random.normal(0, sigma, zSample.size)
zSample = zSample + np.random.normal(0, sigma, zSample.size)


zValidationSample = np.random.choice(zCoords, 200)
yValidationSample = np.abs(0.5 - zValidationSample) * np.cos(
    8 * np.pi * zValidationSample
) + np.random.normal(0, sigma, zValidationSample.size)
xValidationSample = np.abs(0.5 - zValidationSample) * np.sin(
    8 * np.pi * zValidationSample
) + np.random.normal(0, sigma, zValidationSample.size)
zValidationSample = zValidationSample + np.random.normal(
    0, sigma, zValidationSample.size
)

zTestSample = np.random.choice(zCoords, 200)
yTestSample = np.abs(0.5 - zTestSample) * np.cos(
    8 * np.pi * zTestSample
) + np.random.normal(0, sigma, zTestSample.size)
xTestSample = np.abs(0.5 - zTestSample) * np.sin(
    8 * np.pi * zTestSample
) + np.random.normal(0, sigma, zTestSample.size)
zTestSample = zTestSample + np.random.normal(0, sigma, zTestSample.size)

### Plot data

In [ ]:
threeDimensionalAxes = mplplot.figure(figsize=(7, 7)).add_subplot(
    1, 1, 1, projection="3d"
)

# plot the points
threeDimensionalAxes.plot(
    xs=xSample,
    ys=ySample,
    zs=zSample,
    marker="o",
    ls="none",
    label="Training data",
)
threeDimensionalAxes.set_xlabel("X", fontsize="x-large")
threeDimensionalAxes.set_ylabel("Y", fontsize="x-large")
threeDimensionalAxes.set_zlabel("Z", fontsize="x-large")
threeDimensionalAxes.legend(fontsize="x-large")

mplplot.tight_layout()

### Construct Neural Network
Construct a simple fully connected network, that will predict X and Y, given Z.
* The input layer has 100 units (neurons).
* There is one hidden layer with 100 units.
* The output layer has 2 units, representing the features that we want to predict (X,Y)

Use the `tensorflow.keras.Sequential` class and pass a `list` of `tensorflow.keras.layers.Dense` instances.
* The input shape (`[1]`) represents the label Z.

In [ ]:
dlmodel = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            input_shape=[1], units=100, activation=tf.nn.relu
        ),
        tf.keras.layers.Dense(units=100, activation=tf.nn.relu),
        tf.keras.layers.Dense(units=2, activation=tf.nn.tanh),
    ]
)

### Compile the model
When compiling the model we specify the optimizer to use, the loss function and any evaluation metrics we would like to compute.

We print an overview of the model using the `summary()` method.

In [ ]:
dlmodel.compile(
    optimizer="adam", loss="mean_squared_error", metrics=["accuracy"]
)

print(dlmodel.summary())

### Prepare the data
We use a `tensorflow.data.Dataset` to pass our training and validation data to the model.

We transform our labels to lie in the range [-1,1].

We then wrap our labels and features together in a `tuple` and use the `from_tensor_slices()` method to initialize our datasets.

In [ ]:
trainingData = ((2 * zSample) - 1, np.array([xSample, ySample]).T)
validationData = (
    (2 * zValidationSample) - 1,
    np.array([xValidationSample, yValidationSample]).T,
)

trainingDataSet = tf.data.Dataset.from_tensor_slices(trainingData)
validationDataSet = tf.data.Dataset.from_tensor_slices(validationData)

Now we set up our datasets to shuffle the data and provide it to our model in batches of 10 examples. We also use the `repeat()` method to specify that the each example can be fed to the model as many times as required.

In [ ]:
batchSize = 10
trainingDataSet = (
    trainingDataSet.repeat().shuffle(zSample.size).batch(batchSize)
)
validationDataSet = (
    validationDataSet.repeat().shuffle(zValidationSample.size).batch(batchSize)
)

### Train the model
Model training is accomplished using the `fit()` method. We'll specify that training should continue for 1000 epochs. The `steps_per_epoch` argument is actually redundant (and soon to be deprecated), but is required for now. Since one step involves processing one batch of data, and one epoch involves processing all of the data, then the number of steps per epoch is simply the total number of data divided by the number of data per batch.

In [ ]:
trainingHistory = dlmodel.fit(
    trainingDataSet,
    epochs=1000,
    steps_per_epoch=zSample.size // batchSize,
    verbose=False,
    validation_data=validationDataSet,
    validation_steps=zValidationSample.size // batchSize,
)

### Make some predictions
We'll pass a dense grid of Z values and ask our trained model to predict X and Y. We'll compare out model with our training and validation data, as well as the true model that was used to generate them in the first place.

Note that we need to scale our Z coordinates in the same way we did when training the model.

In [ ]:
xyPredictions = dlmodel.predict((2 * zCoords) - 1)

In [ ]:
threeDimensionalAxes = mplplot.figure(figsize=(7, 7)).add_subplot(
    1, 1, 1, projection="3d"
)

# plot the training points
threeDimensionalAxes.plot(
    xs=xSample,
    ys=ySample,
    zs=zSample,
    marker="o",
    ls="none",
    alpha=0.5,
    label="Training",
)
# plot the training points
threeDimensionalAxes.plot(
    xs=xValidationSample,
    ys=yValidationSample,
    zs=zValidationSample,
    marker="o",
    ls="none",
    alpha=0.5,
    label="Validation",
)

# plot the model
threeDimensionalAxes.plot(
    xs=xCoords, ys=yCoords, zs=zCoords, ls="--", c="r", label="Model", lw=0.5
)
# plot the fit
threeDimensionalAxes.plot(
    xs=xyPredictions[:, 0],
    ys=xyPredictions[:, 1],
    zs=zCoords,
    ls=":",
    c="g",
    lw=3,
    label="Fit",
)

threeDimensionalAxes.set_xlabel("X", fontsize="x-large")
threeDimensionalAxes.set_ylabel("Y", fontsize="x-large")
threeDimensionalAxes.set_zlabel("Z", fontsize="x-large")
threeDimensionalAxes.legend()

mplplot.tight_layout()

Not bad!